In [2]:
import pandas as pd

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import root_mean_squared_error

In [4]:
#Training data 
dft=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
print("the january data has " +str(len(dft.columns)))

the january data has 19


In [5]:
#calculate duration using guide notebook
dft['duration'] = dft.tpep_dropoff_datetime - dft.tpep_pickup_datetime
dft.duration = dft.duration.apply(lambda td: td.total_seconds() / 60)
print("the standard deviation for january is " + str(dft.duration.std()))

the standard deviation for january is 42.59435124195458


In [6]:
#we drop outliers using the same principle from the notebook
dftf = dft[(dft.duration >= 1) & (dft.duration <= 60)]
prleft=dftf.shape[0]/dft.shape[0]
print("The percentaje left is " + str(prleft*100)+"%")


The percentaje left is 98.1220282212598%


In [8]:
dftf['PU_DO'] = dftf['PULocationID'].apply(str) + '_' + dftf['DOLocationID'].apply(str)
categorical = ['PU_DO']
numerical = ['trip_distance']
dftf[categorical] = dftf[categorical].astype(str)
print(dftf[categorical])
#With the hot encoding we have 2xn matrix

           PU_DO
0        161_141
1         43_237
2         48_238
3          138_7
4         107_79
...          ...
3066761   107_48
3066762   112_75
3066763  114_239
3066764   230_79
3066765  262_143

[3009173 rows x 1 columns]


/tmp/ipykernel_4601/2797385610.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftf['PU_DO'] = dftf['PULocationID'].apply(str) + '_' + dftf['DOLocationID'].apply(str)
/tmp/ipykernel_4601/2797385610.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftf[categorical] = dftf[categorical].astype(str)


In [9]:
train_dicts = dftf[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = dftf[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

5.133258299449133

In [11]:
#Preparing validation data
dfv=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
dfv['duration'] = dfv.tpep_dropoff_datetime - dfv.tpep_pickup_datetime
dfv.duration = dfv.duration.apply(lambda td: td.total_seconds() / 60)
dfv = dfv[(dfv.duration >= 1) & (dfv.duration <= 60)]
dfv['PU_DO'] = dfv['PULocationID'].apply(str) + '_' + dfv['DOLocationID'].apply(str)
dfv[categorical] = dfv[categorical].astype(str)

In [12]:
val_dicts = dfv[categorical + numerical].to_dict(orient='records')


In [13]:
X_val = dv.transform(val_dicts)
y_pred = lr.predict(X_val)
y_val = dfv[target].values
root_mean_squared_error(y_val, y_pred)

5.24756029797959